# TensorRTへの変換

本処理には十分なGPUメモリが必要です。13_annotation.ipynbのプロセスがシャットダウンされているか確認してください。

In [5]:
import Jetson.GPIO as GPIO

BOARD_NAME = GPIO.gpio_pin_data.get_data()[0]

mode_descriptions = {
    "JETSON_NX": ["15W_2CORE", "15W_4CORE", "15W_6CORE", "10W_2CORE", "10W_4CORE"],
    "JETSON_XAVIER": ["MAXN", "MODE_10W", "MODE_15W", "MODE_30W"],
    "JETSON_NANO": ["MAXN", "5W"],
    "JETSON_ORIN": ["MAXN", "MODE_15W", "MODE_30W", "MODE_40W"],
    "JETSON_ORIN_NANO": ["MODE_15W", "MODE_7W"]
}

product_names = {
    "JETSON_NX": "Jetson Xavier NX",
    "JETSON_XAVIER": "Jetson AGX Xavier",
    "JETSON_NANO": "Jetson Nano",
    "JETSON_ORIN": "Jetson AGX Orin",
    "JETSON_ORIN_NANO": "Jetson Orin Nano"
}

# ボードごとのI2Cバス番号と初期Powerモードを定義する
board_settings = {
    "JETSON_NX": (8, 3),
    "JETSON_XAVIER": (8, 2),
    "JETSON_NANO": (1, 0),
    "JETSON_ORIN": (7, 0),
    "JETSON_ORIN_NANO": (7, 0)
}

i2c_busnum, power_mode = board_settings.get(BOARD_NAME, (None, None))
mode_description = mode_descriptions.get(BOARD_NAME, [])
product_name = product_names.get(BOARD_NAME, "未知のボード")

if power_mode is not None and power_mode < len(mode_description):
    mode_str = mode_description[power_mode]
    print("------------------------------------------------------------")
    print(f"{product_name}を認識: I2Cバス番号: {i2c_busnum}, Powerモード: {mode_str}({power_mode})に設定します。")
    print("------------------------------------------------------------")
else:
    print("未知のボードまたは不正なモードです。")

------------------------------------------------------------
Jetson Orin Nanoを認識: I2Cバス番号: 7, Powerモード: MODE_15W(0)に設定します。
------------------------------------------------------------


In [6]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のため電力モードは変更できません。")
else:
    !echo "jetson" | sudo -S nvpmodel -m $power_mode

Docker起動のため電力モードは変更できません。


In [7]:
!echo "jetson" | sudo -S nvpmodel -q

NVPM WARN: power mode is not set!


In [8]:
if (product_name == "Jetson Orin Nano") or (product_name == "Jetson AGX Orin"):
    print("Docker起動のためjetson_clocksは起動できません。")
else:
    !echo "jetson" | sudo -S jetson_clocks

Docker起動のためjetson_clocksは起動できません。


In [9]:
import torch
import torchvision
from packaging import version
import os

CATEGORIES = ['xy','speed']

output_dim = 2 * len(CATEGORIES)
device = torch.device('cuda')

current_path = os.getcwd()

torchvision_version = version.parse(torchvision.__version__)

# torchvision 0.13以降の場合
if torchvision_version >= version.parse("0.13"):
    from torchvision.models.resnet import ResNet18_Weights, resnet18

    model = resnet18(weights=None)  # pretrained=Falseの代わり
    model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
else:
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, output_dim)

model = model.cuda().eval().half()

In [10]:
from torch2trt import torch2trt

In [11]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

In [12]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob
import time
import datetime
import subprocess

last_model_widget = ipywidgets.Dropdown(options=[],description='変換前')
last_model_time_widget = ipywidgets.Label(description='作成日時：')

trt_model_widget = ipywidgets.Text(description='変換後', value="result.pth")
convert_button = ipywidgets.Button(description='TRT Convert')


def last_model_list(change):
    global last_model_widget
    try:
        files = glob.glob('./model/*.pth', recursive=True)
        last_model_widget.options = files
        
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        last_model_time_widget.value = f'作成日時：{s}'
    except:
        last_model_widget.options = []
last_model_list("list")

def change_file(change):
    file = last_model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    last_model_time_widget.value = f'作成日時：{s}'
last_model_widget.observe(change_file, names='value')


def convert_trt(change):
    global model,model_trt,current_path
    try:
        write_log("モデルの変換を開始します。")
        start_time = time.time()
        write_log("変換前: " + str(last_model_widget.value))
        model.load_state_dict(torch.load(last_model_widget.value))
        write_log("TRTへの変換処理中...(時間がかかります)")
        data = torch.zeros((1, 3, 224, 224)).cuda().half()
        model_trt = torch2trt(model, [data], fp16_mode=True)
        write_log("変換後: " + str(trt_model_widget.value))
        path = os.path.join(current_path,"model_trt")
        if not os.path.exists(path):
            subprocess.call(['mkdir', '-p', path])
        torch.save(model_trt.state_dict(), path + "/" + trt_model_widget.value)
        write_log("保存成功")
        end_time = time.time() - start_time
        write_log("---------------------------------")
        write_log("処理時間　　　　　　　　　: " + str(round(end_time,2)) + "秒")
        write_log("変換前のモデル: " + str(last_model_widget.value))
        write_log("変換後のモデル: " + str(trt_model_widget.value))
        write_log("---------------------------------")
    except Exception as e:
        write_log(f"Error:{e}")
    
convert_button.on_click(convert_trt)

In [15]:
import cv2
import glob
from utils import preprocess
import re

IMG_WIDTH = 224
IMG_HEIGHT = 224
LOAD_TASK = ['camera','train','run','interactive']
SKIP = [1,2,3,4,5]

movie_button = ipywidgets.Button(description='動画の作成')
movie_name_widget = ipywidgets.Text(description='動画名',value="run_video_trt")
load_datasets_widget = ipywidgets.Dropdown(options=[], description='dataset')
load_task_widget = ipywidgets.Dropdown(options=LOAD_TASK, description='task')
skip_dropdown = ipywidgets.Dropdown(options=SKIP, description='skip(枚)', index=1)

def change_load_task(change):
    global current_path
    try:
        path = os.path.join(current_path,load_task_widget.value)
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = [f for f in files if f != ".ipynb_checkpoints"]
        dirs = sorted(dirs)
        load_datasets_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        load_datasets_widget.options = []
load_task_widget.observe(change_load_task, names='value')
change_load_task(LOAD_TASK[0])

def extract_numbers(filename):
    matches = re.findall(r'(\d+)', filename)
    if matches and len(matches) >= 3: 
        return int(matches[-1])  
    else:
        return float('inf')

def get_file_names(path):
    filenames = os.listdir(path)
    filenames = [os.path.join(path, file_name) for file_name in filenames]
    imagenames = []

    imagenames = sorted(filenames, key=lambda f: extract_numbers(os.path.basename(f)))
    imagenames = [f for f in imagenames if os.path.splitext(f)[1].lower() == ".jpg"]
    
    return imagenames

def make_movie(change):
    global model_trt,current_path
    
    if not movie_name_widget.value.strip():
        write_log("ファイル名を指定してください。")
        return 
    write_log("動画を作成します。")
    path = os.path.join(current_path,"video")
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    output = path + "/" + movie_name_widget.value + ".mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(30/skip_dropdown.value)
    outfh = cv2.VideoWriter(output, fourcc, fps, (224, 224))
    
    xy_path = os.path.join(current_path, load_task_widget.value, load_datasets_widget.value, "xy")     
    file_list = get_file_names(xy_path)
    
    try:
        res_num = len(file_list)
        skip = skip_dropdown.value
        count = 0
        for file_name in file_list:
            count += 1
            if count % skip == 0:
                img = cv2.imread(file_name)

                image = preprocess(img).half()
                output = model_trt(image).detach().cpu().numpy().flatten()
                result_x = float(output[0])
                result_y = float(output[1])
                result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
                result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))    
                img = cv2.circle(img, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)

                # Speed
                result_speed = output[3]
                result_speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
                if result_speed > 224:
                    result_speed = 244
                elif result_speed < 0:
                    result_speed = 0
                img = cv2.line(img,(218,0),(218,224),(0,0,0),5)
                img = cv2.line(img,(219,224-result_speed),(219,224),(0,140,255),3)
                img = cv2.putText(img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))

                if count%(skip*10) == 0:
                    write_log(f"{str(int(count/skip))}枚目/{str(int(res_num/skip))}枚中を処理中")
                outfh.write(img)
                del img
    finally:
        # エラーが発生しても確実にリソースを解放する
        outfh.release()
        write_log("動画の出力が完了しました。")
        #get_jetson_nano_memory_usage()

movie_button.on_click(make_movie)

In [16]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.TensorRTへの変換】</b> 軽量化、高速化を目的にモデルをTensorRT形式に変換します。')
title2 = ipywidgets.HTML('<b>【2.変換後のモデルの動画評価】</b> 変換後のモデルで動画を作成し評価します。')

convert_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([last_model_widget,last_model_time_widget]),
    ipywidgets.HBox([trt_model_widget]),
    convert_button,
    process_widget,
    separator,
    title2,
    ipywidgets.HBox([load_datasets_widget,load_task_widget]),
    ipywidgets.HBox([movie_button,movie_name_widget,skip_dropdown]),
    process_widget
])
display(convert_widget)